In [3]:

import sys
import numpy as np
import cv2
import matplotlib.pyplot as plt
import random


!pip install opencv-python

In [5]:
nemo = cv2.imread('nemo.jpg')
h,w=nemo.shape[:2]
print(h,w)
dsize=h,w
src=cv2.imread('junekoo.jpg')
src=cv2.resize(src,dsize)
print(src.shape)


426 640
(640, 426, 3)


In [6]:
if src is None:

    print('Image load failed!')

    sys.exit()

배경과 전경은 무조건 1해 65열 짜리로 만들어 줍니다. 그리고 float64로 한뒤 이것을 계속 업데이트 하여 사용할 겁니다.

In [7]:
mask = np.zeros(src.shape[:2], np.uint8)  # 마스크

bgdModel = np.zeros((1, 65), np.float64)  # 배경 모델

fgdModel = np.zeros((1, 65), np.float64)  # 전경 모델

In [8]:
rc = cv2.selectROI(src)

In [9]:
cv2.grabCut(src, mask, rc, bgdModel, fgdModel, 1, cv2.GC_INIT_WITH_RECT)

(array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 array([[ 3.74439185e-01,  1.13127656e-01,  2.44488304e-01,
          6.23364546e-02,  2.05608401e-01,  2.40509650e+02,
          2.05306630e+02,  1.92697571e+02,  1.59443551e+02,
          1.20840359e+02,  1.13611500e+02,  2.25389714e+02,
          1.83358617e+02,  1.67138935e+02,  1.42329313e+02,
          9.91323599e+01,  8.25716553e+01,  1.88964105e+02,
          1.47552086e+02,  1.38021099e+02,  3.13348904e+01,
          4.05575057e+01,  4.53996119e+01,  4.05575057e+01,
          6.03088868e+01,  7.08364623e+01,  4.53996119e+01,
          7.08364623e+01,  8.96474825e+01,  3.72102607e+02,
          1.95877106e+02, -1.10564847e+02,  1.95877106e+02,
          1.18385665e+02, -3.38688856e+01, -1.10564847e+02,
         -3.38688856e+01,  1.15298346e+02,  5.55341992

In [10]:
mask2 = np.where((mask == 0) | (mask == 2), 0, 1).astype('uint8')

dst = src * mask2[:, :, np.newaxis]

In [11]:
cv2.imshow('dst', dst)

In [12]:
def on_mouse(event, x, y, flags, param):

    #왼쪽버튼을 누르면 포워드

    if event == cv2.EVENT_LBUTTONDOWN:

        cv2.circle(dst, (x, y), 3, (255, 0, 0), -1)

        #파란색

        cv2.circle(mask, (x, y), 3, cv2.GC_FGD, -1)

        cv2.imshow('dst', dst)

    elif event == cv2.EVENT_RBUTTONDOWN:

        #오른쪽버튼을 누르면 백그라운드

        cv2.circle(dst, (x, y), 3, (0, 0, 255), -1)

        cv2.circle(mask, (x, y), 3, cv2.GC_BGD, -1)

        cv2.imshow('dst', dst)

    elif event == cv2.EVENT_MOUSEMOVE:

        #왼쪽버튼 눌른상태에서 움직이면 포워드

        if flags & cv2.EVENT_FLAG_LBUTTON:

            cv2.circle(dst, (x, y), 3, (255, 0, 0), -1)

            cv2.circle(mask, (x, y), 3, cv2.GC_FGD, -1)

            cv2.imshow('dst', dst)

        elif flags & cv2.EVENT_FLAG_RBUTTON:

            #오른쪽버튼 눌른상태에서 움직이면 백그라운드

            cv2.circle(dst, (x, y), 3, (0, 0, 255), -1)

            cv2.circle(mask, (x, y), 3, cv2.GC_BGD, -1)

            cv2.imshow('dst', dst)

마우스 이벤트를 받기 위해서 콜백함수를 추가하였습니다. 

In [13]:
cv2.setMouseCallback('dst', on_mouse)

In [14]:
while True:

    key = cv2.waitKey()

    if key == 13:  # ENTER키를 누르면

        # 사용자가 지정한 전경/배경 정보를 활용하여 영상 분할

        #엔터면 그랙컷 다시호출
        #마스크 입력을 한번 더줘서 결과를 출력해 줍니다.

        cv2.grabCut(src, mask, rc, bgdModel, fgdModel, 1, cv2.GC_INIT_WITH_MASK)

        mask2 = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')

        dst = src * mask2[:, :, np.newaxis]

        cv2.imshow('dst', dst)

    elif key == 27:

        break

In [15]:
cv2.destroyAllWindows()